Final project
===
Anwendungsseminar: Experimentalpsychologische Daten analysieren mit R

In [28]:
library(data.table)
library(ggplot2)

filelist <- list.files(path = "final_data", pattern = "*.txt", full.names = TRUE)
datalist <- lapply(filelist, function(x)read.table(x, header=T)) 
df <- do.call("rbind", datalist)
setDT(df) # Convert data.frame to data.table

In [29]:
df

cueType,targetValidity,CueDistractorPosition1,cueDistractor2_Position,cueDistractor3_Position,InhibitionDistractor,Target,TargetColor,gap_topleft,gap_topright,...,corrResp,key_responseTarget.corr_raw,key_responseTarget.rt_raw,sex,hand,key_responseTarget.keys_raw,sbj,order,age,instruction
match,valid,topleft,bottomright,bottomleft,topleft,topright,blue,left,right,...,right,0,,m,r,,1,215,22,negative
match,valid,topleft,bottomright,bottomleft,topleft,topright,blue,down,right,...,right,1,"0,618039072",m,r,'right',1,1249,22,negative
match,valid,topleft,bottomright,bottomleft,bottomright,topright,blue,down,left,...,left,0,"0,729897082",m,r,'up',1,900,22,negative
match,invalid,topleft,bottomright,bottomleft,bottomleft,topleft,blue,left,down,...,left,0,"0,938588321",m,r,'down',1,554,22,negative
match,invalid,topleft,bottomright,bottomleft,bottomleft,topleft,blue,down,left,...,down,1,"0,772500098",m,r,'down',1,110,22,negative
match,invalid,topleft,bottomright,bottomleft,topright,topleft,blue,up,down,...,up,1,"0,786968172",m,r,'up',1,691,22,negative
match,invalid,topleft,bottomright,bottomleft,topright,bottomright,blue,left,down,...,up,1,"0,506554425",m,r,'up',1,1333,22,negative
match,invalid,topleft,bottomright,bottomleft,topright,bottomright,blue,down,right,...,up,1,"0,591145694",m,r,'up',1,663,22,negative
match,invalid,topleft,bottomright,bottomleft,bottomleft,bottomright,blue,left,right,...,down,1,"0,543891609",m,r,'down',1,426,22,negative
match,invalid,topleft,bottomright,bottomleft,bottomright,bottomleft,blue,up,down,...,left,1,"0,535076678",m,r,'left',1,1149,22,negative
